# Notebook for generating the submission csv.
-------------------------------------------------------------------------------------------------------------------
# Technology used: Tensorflow core

I start with the usual utility cells

In [1]:
# packages used for processing:
import cPickle as pickle # for pickling the processed data
import matplotlib.pyplot as plt # for visualization
import numpy as np # numerical computations

# for operating system related stuff
import os
import sys # for memory usage of objects
from subprocess import check_output

# pandas for extracting data from csv file
import pandas as pd

# the boss of deep learning frameworks
import tensorflow as tf

# to plot the images inline
%matplotlib inline

In [2]:
# Input data files are available in the "../Data/" directory.

def exec_command(cmd):
    '''
        function to execute a shell command and see it's 
        output in the python console
        @params
        cmd = the command to be executed along with the arguments
              ex: ['ls', '../input']
    '''
    print(check_output(cmd).decode("utf8"))

In [3]:
# check the structure of the project directory
exec_command(['ls', '..'])

Data
LICENSE
Models
README.md
Scripts
submissions :D



In [25]:
''' Set the constants for the script '''

# various paths of the files
data_path = "../Data" # the data path
base_model_path = "../Models"

data_files = {
    "train": os.path.join(data_path, "train.csv"),
    "test": os.path.join(data_path, "test.csv")
}

base_model_path = '../Models'

plug_and_play_data_file_path = os.path.join(data_path, "plug_and_play_for_ANN.pickle")

# constants:
(train_size, dev_size, test_size) = (0.9, 0.05, 0.05) # values are unit ratios
no_of_features = 57
no_of_itreations = 10000 
batch_size = 512
checkpoint_factor = 50
lr = 1e-6

In [5]:
# function to unpickle the given file and load the obj back into the python environment
def unPickleIt(pickle_path): # might throw the file not found exception
    '''
        function to unpickle the object from the given path
        @param
        pickle_path => the path where the pickle file is located
        @return => the object extracted from the saved path
    '''

    with open(pickle_path, 'rb') as dumped_pickle:
        obj = pickle.load(dumped_pickle)

    return obj # return the unpickled object

# Load the data from the test.csv file to generate predictions from them

In [11]:
# load the means and variances from the plug_and_play file
dat_dict = unPickleIt(plug_and_play_data_file_path)
variances = dat_dict['variances']

In [12]:
# delete the dat_dict to free up resources
del dat_dict

In [13]:
# check the shapes of theses two vals
print variances.shape

(57, 1)


In [14]:
# load the data from the test.csv file
raw_data = pd.read_csv(data_files['test'])

In [15]:
# print a few rows of the raw_data
raw_data.head(10)

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,0,1,8,1,0,0,1,0,0,...,1,1,1,12,0,1,1,0,0,1
1,1,4,2,5,1,0,0,0,0,1,...,2,0,3,10,0,0,1,1,0,1
2,2,5,1,3,0,0,0,0,0,1,...,4,0,2,4,0,0,0,0,0,0
3,3,0,1,6,0,0,1,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,4,5,1,7,0,0,0,0,0,1,...,4,0,0,4,0,1,1,0,0,1
5,5,0,1,6,0,0,1,0,0,0,...,8,1,4,9,1,0,1,0,1,0
6,6,0,1,3,0,0,0,1,0,0,...,2,0,4,6,1,1,0,0,0,0
7,8,0,1,0,0,0,1,0,0,0,...,3,1,4,9,0,1,0,0,0,0
8,10,0,1,7,0,0,0,1,0,0,...,5,1,4,6,0,0,1,0,0,0
9,11,1,1,6,0,0,0,0,0,1,...,6,1,6,10,0,1,1,0,0,0


## Note that the target column is missing! since these are the vals for which predictions are to be given

In [16]:
# check the number of test examples for which predictions are to be generated.
n_test_examples = raw_data['id'].count()
print "Total test examples to be predicted: " + str(n_test_examples)

Total test examples to be predicted: 892816


In [17]:
# transform this new data to normalize it using the earlier means and variances
def normalize_data_frame(data, variances):
    '''
        function to normalize the pandas dataframe and convert it into a numpy array
        @param
        data => the pandas dataframe
        variances => the variances array for variance correction
        @return => features array
    '''
    
    # create an empty data structure to hold all the data
    features = np.ndarray(shape = (len(data.columns) - 1, data.id.count()))
    
    # iterate over all the columns and insert their slices into the features array after normalizing them
    count = 0; # start the counter from 0 and perform the required stuff
    for column in data.columns[1:]:
        feature_slice = np.array(data[column]).reshape(1, -1) # carve out the feature slice
        variance = variances[count]
        
        feature_slice = feature_slice + 1 # change of origin
        feature_slice = feature_slice / variance # variance normalization
        
        # add the slice to the features vector
        features[count, :] = feature_slice
        
        # do not forget to increment the counter
        count += 1
    
    return features

In [19]:
test_data = normalize_data_frame(raw_data, variances)

In [20]:
print "Shape of test data: " + str(test_data.shape)

Shape of test data: (57, 892816)


In [22]:
test_data[:10, :2]

array([[  2.54102959e-01,   1.27051480e+00],
       [  4.52811810e+00,   6.79217715e+00],
       [  1.23465964e+00,   8.23106425e-01],
       [  8.21842224e+00,   8.21842224e+00],
       [  5.48176031e-01,   5.48176031e-01],
       [  4.18919668e+00,   4.18919668e+00],
       [  1.04730173e+01,   5.23650864e+00],
       [  7.29654360e+00,   7.29654360e+00],
       [  6.62399692e+00,   1.32479938e+01],
       [  2.68213551e+03,   2.68213551e+03]])

# The data has been properly set up. I can now proceed further with the predictions generation.

In [23]:
# the num_units in each layer of the feed_forward neural network
layer_dims = [512, 512, 512, 512, 512, 512, 512, 512, 2]

In [26]:
tf.reset_default_graph()


# In[17]:


# scoped as Inputs
with tf.variable_scope("Input"):
   # define the placeholders for the input data
   # placeholder for feeding in input data batch
   input_X = tf.placeholder(tf.float32, shape=(None, no_of_features), name="Input_features")
   labels_Y = tf.placeholder(tf.int32, shape=(None,), name="Ideal_labels") # placeholder for the labels
   one_hot_encoded_labels_Y = tf.one_hot(labels_Y, depth=2, axis=1, name="One_hot_label_encoder")


# In[18]:


one_hot_encoded_labels_Y


# In[19]:


# scoped as model:
with tf.variable_scope("Deep_Neural_Network"):
    # define the layers for the neural network.
    with tf.name_scope("Encoder"):
        ''' This is The forward-backward neural network with abs activation function '''
        # layer 1 => 
        fwd_lay1 = tf.layers.dense(input_X, layer_dims[0], activation=tf.abs, name="layer_1")
        # layer 2 =>
        fwd_lay2 = tf.layers.dense(fwd_lay1, layer_dims[1], activation=tf.abs, name="layer_2")
        # layer 3 =>
        fwd_lay3 = tf.layers.dense(fwd_lay2, layer_dims[2], activation=tf.abs, name="layer_3")
        # layer 4 =>
        fwd_lay4 = tf.layers.dense(fwd_lay3, layer_dims[3], activation=tf.abs, name="layer_4")
        # layer 5 =>
        fwd_lay5 = tf.layers.dense(fwd_lay4, layer_dims[4], activation=tf.abs, name="layer_5")
        # layer 6 =>
        fwd_lay6 = tf.layers.dense(fwd_lay5, layer_dims[5], activation=tf.abs, name="layer_6")
        # layer 7 =>
        fwd_lay7 = tf.layers.dense(fwd_lay6, layer_dims[6], activation=tf.abs, name="layer_7")
        # layer 8 =>
        fwd_lay8 = tf.layers.dense(fwd_lay7, layer_dims[7], activation=tf.abs, name="layer_8")
        # layer 9 =>
        fwd_lay9 = tf.layers.dense(fwd_lay8, layer_dims[8], activation=tf.abs, name="layer_9")
        
    ''' Separately record all the activations as histograms '''
    # recording the summaries to visualize separately
    fwd_lay1_summary = tf.summary.histogram("fwd_lay1_summary", fwd_lay1)
    fwd_lay2_summary = tf.summary.histogram("fwd_lay2_summary", fwd_lay2)
    fwd_lay3_summary = tf.summary.histogram("fwd_lay3_summary", fwd_lay3)
    fwd_lay4_summary = tf.summary.histogram("fwd_lay4_summary", fwd_lay4)
    fwd_lay5_summary = tf.summary.histogram("fwd_lay5_summary", fwd_lay5)
    fwd_lay6_summary = tf.summary.histogram("fwd_lay6_summary", fwd_lay6)
    fwd_lay7_summary = tf.summary.histogram("fwd_lay7_summary", fwd_lay7)
    fwd_lay8_summary = tf.summary.histogram("fwd_lay8_summary", fwd_lay8)
    fwd_lay9_summary = tf.summary.histogram("fwd_lay9_summary", fwd_lay9)


# In[20]:


with tf.variable_scope("", reuse=True):
    # bring out all the weights from the network
    lay_1_wts = tf.get_variable("Deep_Neural_Network/layer_1/kernel")
    lay_2_wts = tf.get_variable("Deep_Neural_Network/layer_2/kernel")
    lay_3_wts = tf.get_variable("Deep_Neural_Network/layer_3/kernel")
    lay_4_wts = tf.get_variable("Deep_Neural_Network/layer_4/kernel")
    lay_5_wts = tf.get_variable("Deep_Neural_Network/layer_5/kernel")
    lay_6_wts = tf.get_variable("Deep_Neural_Network/layer_6/kernel")
    lay_7_wts = tf.get_variable("Deep_Neural_Network/layer_7/kernel")
    lay_8_wts = tf.get_variable("Deep_Neural_Network/layer_8/kernel")
    lay_9_wts = tf.get_variable("Deep_Neural_Network/layer_9/kernel")
    
    lay_1_biases = tf.get_variable("Deep_Neural_Network/layer_1/bias")
    lay_2_biases = tf.get_variable("Deep_Neural_Network/layer_2/bias")
    lay_3_biases = tf.get_variable("Deep_Neural_Network/layer_3/bias")
    lay_4_biases = tf.get_variable("Deep_Neural_Network/layer_4/bias")
    lay_5_biases = tf.get_variable("Deep_Neural_Network/layer_5/bias")
    lay_6_biases = tf.get_variable("Deep_Neural_Network/layer_6/bias")
    lay_7_biases = tf.get_variable("Deep_Neural_Network/layer_7/bias")
    lay_8_biases = tf.get_variable("Deep_Neural_Network/layer_8/bias")
    lay_9_biases = tf.get_variable("Deep_Neural_Network/layer_9/bias")


# In[21]:


lay_1_wts, lay_8_wts, lay_9_wts, lay_1_biases, lay_8_biases, lay_9_biases


# In[22]:


y_back_in = fwd_lay9
y_back_in


# In[23]:


with tf.name_scope("Decoder"):
        lay_0_biases = tf.get_variable("layer_0/bias", shape=(no_of_features, ))
    
        # layer 1 => 
        bwd_lay1 = tf.abs(tf.matmul(y_back_in, tf.transpose(lay_9_wts)) + lay_8_biases)
        # layer 2 => 
        bwd_lay2 = tf.abs(tf.matmul(bwd_lay1, tf.transpose(lay_8_wts)) + lay_7_biases)
        # layer 3 => 
        bwd_lay3 = tf.abs(tf.matmul(bwd_lay2, tf.transpose(lay_7_wts)) + lay_6_biases)
        # layer 4 => 
        bwd_lay4 = tf.abs(tf.matmul(bwd_lay3, tf.transpose(lay_6_wts)) + lay_5_biases)
        # layer 5 => 
        bwd_lay5 = tf.abs(tf.matmul(bwd_lay4, tf.transpose(lay_5_wts)) + lay_4_biases)
        # layer 6 => 
        bwd_lay6 = tf.abs(tf.matmul(bwd_lay5, tf.transpose(lay_4_wts)) + lay_3_biases)
        # layer 7 => 
        bwd_lay7 = tf.abs(tf.matmul(bwd_lay6, tf.transpose(lay_3_wts)) + lay_2_biases)
        # layer 8 => 
        bwd_lay8 = tf.abs(tf.matmul(bwd_lay7, tf.transpose(lay_2_wts)) + lay_1_biases)
        # layer 9 => 
        bwd_lay9 = tf.abs(tf.matmul(bwd_lay8, tf.transpose(lay_1_wts)) + lay_0_biases)


# In[24]:


x_back_out = bwd_lay9
x_back_out, input_X


# In[25]:


# function to compute the directional cosines of the input values
def directional_cosines(X):
    ''' 
        calculate the directional cosines of the inputs
    '''
    square = tf.square(X)
    sum_square = tf.reduce_sum(square, axis=1, keep_dims=True)
    dcs = X / tf.sqrt(sum_square)
    
    # return the directional cosines:
    return dcs


# In[26]:


# scoped as predictions
with tf.variable_scope("Prediction"):
    prediction = directional_cosines(y_back_in)


# In[27]:


# scoped as loss
with tf.variable_scope("Loss"):
    # define the forward loss
    fwd_loss = tf.reduce_mean(tf.abs(prediction - one_hot_encoded_labels_Y))
    
    # define the reverse loss
    rev_loss = tf.reduce_mean(tf.abs(x_back_out - input_X))
    
    total_loss = fwd_loss + rev_loss
        
    # record the loss summary:
    tf.summary.scalar("Fwd_loss", fwd_loss)
    tf.summary.scalar("Bwd_loss", rev_loss)
    tf.summary.scalar("Tot_loss", total_loss)


# In[28]:


# scoped as train_step
with tf.variable_scope("Train_Step"):
    # define the optimizer and the train_step:
    optimizer = tf.train.AdamOptimizer(learning_rate=lr) # this has been manually tuned
    train_step = optimizer.minimize(total_loss, name="train_step")


# In[29]:


# scoped as init operation
with tf.variable_scope("Init"):
    init_op = tf.global_variables_initializer()


# In[30]:


# scoped as summaries
with tf.variable_scope("Summary"):
    all_summaries = tf.summary.merge_all()

In [27]:
def generate_predictions(dataX, exec_graph, model_name):
    '''
        Function to run the trained model and generate predictions for the given data
        @param 
        dataX => The data to be used for accuracy calculation
        exec_graph => the Computation graph to be used
        model_name => the model to restore the weights from
        @return => predictions array returned
    '''
    
    # the number of examples in the dataset
    no_of_examples = dataX.shape[-1]
    
    with tf.Session(graph=exec_graph) as sess:
        
        # The saver object for saving and loading the model
        saver = tf.train.Saver(max_to_keep=2)
        
        # the model must exist and you must be able to restore the weights
        model_path = os.path.join(base_model_path, model_name)
        assert os.path.isfile(os.path.join(model_path, "checkpoint")), "Model doesn't exist"
        
        saver.restore(sess, tf.train.latest_checkpoint(model_path))
        
        # compute the predictions given out by model
        preds = sess.run(prediction, feed_dict={input_X: dataX.T})
        
    # return the so calculated accuracy:
    return preds

In [ ]:
''' 
    WARNING! WARNING! WARNING!
    Keep an eye on the htop meter while executing this cell. The machine might freeze momentarily if it
    is a low end machine.
'''

# get the predictions for the test_data.
model_name = os.path.join(base_model_path, "Model2")
predictions = generate_predictions(test_data, tf.get_default_graph(), model_name=model_name)

INFO:tensorflow:Restoring parameters from ../Models/../Models/Model2/Model2-250


In [ ]:
# read the sample_submission.csv file and extract the ids from it.
sample_submission = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

In [ ]:
ids = np.array(sample_submission['id'])
print "The ids column has a shape: " + str(ids.shape)

In [ ]:
# print the shape of the generated predictions
print "The generated predictions have the shape: " + str(np.squeeze(predictions).shape)

In [ ]:
# lets quickly write the function to generate the subimssion csv file
def generate_submission_file(preds, save_path, model_name):
    '''
        function to generate the submission file. 
        @param
        preds => the predictions to be written to the file
        model_name => the model used for this generation 
        save_path => the path where the file needs to be saved
        @return => None (check the save path where the file is saved)
    '''
    save_file = save_path + '_' + model_name + '.csv'
    
    with open(save_file, 'w') as submission:
        # write the header to the file
        submission.write("id,target\n")
        for (idx, prediction) in zip(ids, np.squeeze(predictions)):
            line = str(idx) + ',' + str(prediction) + '\n'
            submission.write(line) # write the line to the file
    
    # print a feedback statement to notify the required file generation
    print "The file has been generated at: " + save_file

In [ ]:
# use the above function to generate the submission file
generate_submission_file(predictions, os.path.join(data_path, "submission"), "Model1")